In [3]:
import numpy as np

class XavierInitializer:
    def __init__(self):
        pass
    
    def initialize(self, shape):
        # Xavier initialization: std = sqrt(1 / n), where n is the number of input units
        n_in = shape[0]
        stddev = np.sqrt(1. / n_in)
        return np.random.normal(0, stddev, size=shape)

class SimpleConv1d:
    def __init__(self, input_size, filter_size, initializer=XavierInitializer()):
        self.input_size = input_size  # Length of the input sequence
        self.filter_size = filter_size  # Size of the filter
        self.initializer = initializer
        self.weights = self.initializer.initialize((filter_size, 1))  # Filter weights (1 channel)
        self.bias = np.zeros(1)  # Bias (scalar)
        self.output_size = input_size - filter_size + 1  # Output size, assuming no padding and stride=1

    def forward(self, X):
        self.X = X
        self.output = np.zeros(self.output_size)

        # Perform convolution
        for i in range(self.output_size):
            self.output[i] = np.sum(self.X[i:i + self.filter_size] * self.weights.flatten()) + self.bias
        return self.output

    def backward(self, d_output, learning_rate):
        # Initialize gradients
        d_weights = np.zeros_like(self.weights)
        d_bias = 0
        d_input = np.zeros_like(self.X)

        # Compute gradients for weights, bias, and input
        for i in range(self.output_size):
            # Reshape input slice to match the shape of weights (filter_size, 1)
            input_slice = self.X[i:i + self.filter_size].reshape(-1, 1)
            d_weights += d_output[i] * input_slice
            d_bias += d_output[i]
            # Compute gradient for input (error propagation)
            for s in range(self.filter_size):
                if 0 <= i - s < len(d_input):
                    d_input[i - s] += d_output[i] * self.weights[s]

        # Update weights and bias
        self.weights -= learning_rate * d_weights
        self.bias -= learning_rate * d_bias

        return d_input  # Return the error to propagate backwards


In [4]:
# Example input (1D array)
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Create a SimpleConv1d layer with input size 10, filter size 3
conv_layer = SimpleConv1d(input_size=10, filter_size=3)

# Forward pass
output = conv_layer.forward(X)
print("Output:", output)

# Backward pass (dummy gradient)
d_output = np.random.randn(len(output))  # Random gradient for demonstration
conv_layer.backward(d_output, learning_rate=0.01)


Output: [ -2.74465739  -4.16837041  -5.59208343  -7.01579645  -8.43950947
  -9.86322249 -11.28693551 -12.71064853]


C:\Users\Mercy\AppData\Local\Temp\ipykernel_14168\3194059677.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.output[i] = np.sum(self.X[i:i + self.filter_size] * self.weights.flatten()) + self.bias
C:\Users\Mercy\AppData\Local\Temp\ipykernel_14168\3194059677.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  d_input[i - s] += d_output[i] * self.weights[s]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
def calculate_output_size(N_in, P, F, S):
    """
    Calculate the output size of a 1D convolution layer.

    Parameters:
    - N_in: Input size (number of features)
    - P: Padding size
    - F: Filter size
    - S: Stride size

    Returns:
    - N_out: Output size (number of features)
    """
    N_out = (N_in + 2 * P - F) // S + 1
    return N_out
# Example parameters
N_in = 10  # Input size
P = 1      # Padding size
F = 3      # Filter size
S = 1      # Stride size

# Calculate the output size
N_out = calculate_output_size(N_in, P, F, S)
print("Output size:", N_out)



Output size: 10


In [13]:
import numpy as np

# Input, weights, and bias
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
b = np.array([1])

# Forward pass
a = np.empty(2)
indexes0 = np.array([0, 1, 2]).astype(int)  # Use 'int' instead of 'np.int'
indexes1 = np.array([1, 2, 3]).astype(int)  # Use 'int' instead of 'np.int'

a[0] = np.sum(x[indexes0] * w) + b
a[1] = np.sum(x[indexes1] * w) + b

print("Forward pass output:", a)

# Backpropagation error (delta)
delta_a = np.array([10, 20])

# Calculate gradients
delta_b = np.sum(delta_a)
delta_w = np.zeros_like(w)
delta_x = np.zeros_like(x)

# Gradients for weights and input
for i in range(len(delta_a)):
    delta_w += delta_a[i] * x[indexes0[i]:indexes0[i] + len(w)]  # Accumulate gradients for weights
    delta_x[indexes0[i]:indexes0[i] + len(w)] += delta_a[i] * w  # Error gradient w.r.t. input

print("Backward pass results:")
print("delta_b:", delta_b)
print("delta_w:", delta_w)
print("delta_x:", delta_x)


Forward pass output: [35. 50.]
Backward pass results:
delta_b: 30
delta_w: [ 50  80 110]
delta_x: [ 30 110 170 140]


C:\Users\Mercy\AppData\Local\Temp\ipykernel_14168\2896273398.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a[0] = np.sum(x[indexes0] * w) + b
C:\Users\Mercy\AppData\Local\Temp\ipykernel_14168\2896273398.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a[1] = np.sum(x[indexes1] * w) + b


In [16]:
import numpy as np

class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size):
        # Initialize filters and bias
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        
        # Filters: output_channels x input_channels x filter_size
        self.w = np.ones((output_channels, input_channels, filter_size))  # Example: all ones for simplicity
        
        # Bias: output_channels (initialized as float64)
        self.b = np.array([1, 2, 3], dtype=np.float64)  # Ensure bias is float64
        
    def forward(self, x):
        """
        Forward pass through the Conv1d layer
        x: input data with shape (input_channels, feature_length)
        Returns: output data with shape (output_channels, output_length)
        """
        output_length = x.shape[1] - self.filter_size + 1
        output = np.zeros((self.output_channels, output_length), dtype=np.float64)
        
        # Perform convolution for each output channel
        for i in range(self.output_channels):
            for j in range(output_length):
                # Convolve the filter with the input x
                for c in range(self.input_channels):
                    output[i, j] += np.sum(x[c, j:j + self.filter_size] * self.w[i, c])
                # Add bias for the output channel
                output[i, j] += self.b[i]
        
        return output
    
    def backward(self, x, delta_a, learning_rate=0.01):
        """
        Backward pass through the Conv1d layer
        x: input data with shape (input_channels, feature_length)
        delta_a: gradient of the loss with respect to the output, shape (output_channels, output_length)
        learning_rate: learning rate for gradient descent
        """
        delta_w = np.zeros_like(self.w, dtype=np.float64)
        delta_b = np.zeros_like(self.b, dtype=np.float64)
        
        # Ensure delta_x is initialized with the correct data type
        delta_x = np.zeros_like(x, dtype=np.float64)
        
        # Gradients for weights, biases, and input
        for i in range(self.output_channels):
            for j in range(delta_a.shape[1]):
                delta_b[i] += delta_a[i, j]
                for c in range(self.input_channels):
                    delta_w[i, c] += delta_a[i, j] * x[c, j:j + self.filter_size].sum()
                    delta_x[c, j:j + self.filter_size] += delta_a[i, j] * self.w[i, c]
        
        # Update weights and biases using gradient descent
        self.w -= learning_rate * delta_w
        self.b -= learning_rate * delta_b
        
        return delta_x


# Example usage:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])  # Input: 2 channels, 4 features
w = np.ones((3, 2, 3))  # Filters: 3 output channels, 2 input channels, filter size 3
b = np.array([1, 2, 3])  # Biases: 3 output channels

# Create Conv1d layer
conv_layer = Conv1d(input_channels=2, output_channels=3, filter_size=3)

# Forward pass
output = conv_layer.forward(x)
print("Forward pass output:")
print(output)

# Backpropagation (dummy delta_a for demonstration)
delta_a = np.array([[10, 20], [15, 25], [30, 35]])  # Dummy gradient of loss with respect to output
delta_x = conv_layer.backward(x, delta_a, learning_rate=0.01)

print("Backward pass:")
print("Gradient w.r.t input:")
print(delta_x)


Forward pass output:
[[16. 22.]
 [17. 23.]
 [18. 24.]]
Backward pass:
Gradient w.r.t input:
[[ 55. 135. 135.  80.]
 [ 55. 135. 135.  80.]]


In [17]:
import numpy as np

class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size, padding=0):
        # Initialize filters and bias
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        self.padding = padding
        
        # Filters: output_channels x input_channels x filter_size
        self.w = np.ones((output_channels, input_channels, filter_size))  # Example: all ones for simplicity
        
        # Bias: output_channels (initialized as float64)
        self.b = np.array([1, 2, 3], dtype=np.float64)  # Ensure bias is float64
        
    def pad_input(self, x):
        """
        Apply padding to the input array using np.pad().
        Default padding is zero-padding.
        """
        return np.pad(x, ((0, 0), (self.padding, self.padding)), mode='constant', constant_values=0)
    
    def forward(self, x):
        """
        Forward pass through the Conv1d layer with padding
        x: input data with shape (input_channels, feature_length)
        Returns: output data with shape (output_channels, output_length)
        """
        # Apply padding to the input
        x_padded = self.pad_input(x)
        output_length = x_padded.shape[1] - self.filter_size + 1
        output = np.zeros((self.output_channels, output_length), dtype=np.float64)
        
        # Perform convolution for each output channel
        for i in range(self.output_channels):
            for j in range(output_length):
                # Convolve the filter with the padded input x
                for c in range(self.input_channels):
                    output[i, j] += np.sum(x_padded[c, j:j + self.filter_size] * self.w[i, c])
                # Add bias for the output channel
                output[i, j] += self.b[i]
        
        return output
    
    def backward(self, x, delta_a, learning_rate=0.01):
        """
        Backward pass through the Conv1d layer with padding
        x: input data with shape (input_channels, feature_length)
        delta_a: gradient of the loss with respect to the output, shape (output_channels, output_length)
        learning_rate: learning rate for gradient descent
        """
        # Apply padding to the input for backpropagation
        x_padded = self.pad_input(x)
        
        delta_w = np.zeros_like(self.w, dtype=np.float64)
        delta_b = np.zeros_like(self.b, dtype=np.float64)
        
        # Ensure delta_x is initialized with the correct data type
        delta_x = np.zeros_like(x_padded, dtype=np.float64)
        
        # Gradients for weights, biases, and input
        for i in range(self.output_channels):
            for j in range(delta_a.shape[1]):
                delta_b[i] += delta_a[i, j]
                for c in range(self.input_channels):
                    delta_w[i, c] += delta_a[i, j] * x_padded[c, j:j + self.filter_size].sum()
                    delta_x[c, j:j + self.filter_size] += delta_a[i, j] * self.w[i, c]
        
        # Remove padding from the gradient w.r.t input (delta_x)
        delta_x = delta_x[:, self.padding:-self.padding] if self.padding > 0 else delta_x
        
        # Update weights and biases using gradient descent
        self.w -= learning_rate * delta_w
        self.b -= learning_rate * delta_b
        
        return delta_x


# Example usage:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])  # Input: 2 channels, 4 features
w = np.ones((3, 2, 3))  # Filters: 3 output channels, 2 input channels, filter size 3
b = np.array([1, 2, 3])  # Biases: 3 output channels

# Create Conv1d layer with padding=1 (zero padding)
conv_layer = Conv1d(input_channels=2, output_channels=3, filter_size=3, padding=1)

# Forward pass
output = conv_layer.forward(x)
print("Forward pass output:")
print(output)

# Backpropagation (dummy delta_a for demonstration)
delta_a = np.array([[10, 20], [15, 25], [30, 35]])  # Dummy gradient of loss with respect to output
delta_x = conv_layer.backward(x, delta_a, learning_rate=0.01)

print("Backward pass:")
print("Gradient w.r.t input:")
print(delta_x)


Forward pass output:
[[ 9. 16. 22. 17.]
 [10. 17. 23. 18.]
 [11. 18. 24. 19.]]
Backward pass:
Gradient w.r.t input:
[[135. 135.  80.   0.]
 [135. 135.  80.   0.]]


In [36]:
import numpy as np

class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size, padding=0):
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        self.padding = padding
        
        # Initialize filters and biases
        self.w = np.ones((output_channels, input_channels, filter_size))  # All filters set to 1 for simplicity
        self.b = np.array([1, 2, 3], dtype=np.float64)  # Biases initialized as floats
        
    def pad_input(self, x):
        """
        Apply padding to the input array using np.pad().
        Default padding is zero-padding.
        """
        return np.pad(x, ((0, 0), (self.padding, self.padding), (0, 0)), mode='constant', constant_values=0)
    
    def forward(self, x):
        """
        Forward pass through the Conv1d layer with padding
        x: input data with shape (batch_size, input_channels, feature_length)
        Returns: output data with shape (batch_size, output_channels, output_length)
        """
        # Apply padding to the input
        x_padded = self.pad_input(x)
        
        batch_size, input_channels, input_length = x_padded.shape
        output_length = input_length - self.filter_size + 1  # Ensure output size is calculated correctly
        output = np.zeros((batch_size, self.output_channels, output_length), dtype=np.float64)
        
        # Perform convolution for each output channel, and each sample in the batch
        for i in range(self.output_channels):
            for b in range(batch_size):  # Process each sample in the batch
                for j in range(output_length):  # Iterate over the valid output positions
                    # Convolve the filter with the padded input for the b-th sample
                    for c in range(input_channels):
                        start = j  # Starting index for the slice
                        end = start + self.filter_size  # Ending index for the slice
                        # Adjust the end index if it goes out of bounds
                        if end > input_length + self.padding:
                            end = input_length + self.padding
                        # Perform the convolution operation
                        output[b, i, j] += np.sum(x_padded[b, c, start:end] * self.w[i, c, :end-start])
                    # Add bias for the output channel
                    output[b, i, j] += self.b[i]
        
        return output
    
    def backward(self, x, delta_a, learning_rate=0.01):
        """
        Backward pass through the Conv1d layer with padding
        x: input data with shape (batch_size, input_channels, feature_length)
        delta_a: gradient of the loss with respect to the output, shape (batch_size, output_channels, output_length)
        learning_rate: learning rate for gradient descent
        """
        # Apply padding to the input for backpropagation
        x_padded = self.pad_input(x)
        
        batch_size, input_channels, input_length = x.shape
        delta_w = np.zeros_like(self.w, dtype=np.float64)
        delta_b = np.zeros_like(self.b, dtype=np.float64)
        
        # Initialize delta_x for the backpropagation of each sample in the batch
        delta_x = np.zeros_like(x_padded, dtype=np.float64)
        
        # Gradients for weights, biases, and input (accumulated over the batch)
        for i in range(self.output_channels):
            for b in range(batch_size):  # Process each sample in the batch
                for j in range(delta_a.shape[2]):
                    delta_b[i] += delta_a[b, i, j]
                    for c in range(input_channels):
                        delta_w[i, c] += delta_a[b, i, j] * x_padded[b, c, j:j + self.filter_size].sum()
                        delta_x[b, c, j:j + self.filter_size] += delta_a[b, i, j] * self.w[i, c]
        
        # Remove padding from the gradient w.r.t input (delta_x)
        delta_x = delta_x[:, :, self.padding:-self.padding] if self.padding > 0 else delta_x
        
        # Update weights and biases using gradient descent
        self.w -= learning_rate * delta_w
        self.b -= learning_rate * delta_b
        
        return delta_x

In [39]:
import numpy as np

class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size, padding=0):
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        self.padding = padding
        
        # Initialize filters and biases
        self.w = np.ones((output_channels, input_channels, filter_size))  # All filters set to 1 for simplicity
        self.b = np.array([1, 2, 3], dtype=np.float64)  # Biases initialized as floats
        
    def pad_input(self, x):
        """
        Apply padding to the input array using np.pad().
        Default padding is zero-padding.
        """
        return np.pad(x, ((0, 0), (self.padding, self.padding), (0, 0)), mode='constant', constant_values=0)
    
    def forward(self, x):
        """
        Forward pass through the Conv1d layer with padding
        x: input data with shape (batch_size, input_channels, feature_length)
        Returns: output data with shape (batch_size, output_channels, output_length)
        """
        # Apply padding to the input
        x_padded = self.pad_input(x)
        
        batch_size, input_channels, input_length = x_padded.shape
        output_length = input_length - self.filter_size + 1  # Ensure output size is calculated correctly
        output = np.zeros((batch_size, self.output_channels, output_length), dtype=np.float64)
        
        # Perform convolution for each output channel, and each sample in the batch
        for i in range(self.output_channels):
            for b in range(batch_size):  # Process each sample in the batch
                for j in range(output_length):  # Iterate over the valid output positions
                    # Convolve the filter with the padded input for the b-th sample
                    for c in range(input_channels):
                        start = j  # Starting index for the slice
                        end = start + self.filter_size  # Ending index for the slice
                        # Ensure that the slice doesn't go out of bounds
                        if end > input_length + self.padding:
                            end = input_length + self.padding
                        # Perform the convolution operation
                        output[b, i, j] += np.sum(x_padded[b, c, start:end] * self.w[i, c, :end-start])
                    # Add bias for the output channel
                    output[b, i, j] += self.b[i]
        
        return output
    
    def backward(self, x, delta_a, learning_rate=0.01):
        """
        Backward pass through the Conv1d layer with padding
        x: input data with shape (batch_size, input_channels, feature_length)
        delta_a: gradient of the loss with respect to the output, shape (batch_size, output_channels, output_length)
        learning_rate: learning rate for gradient descent
        """
        # Apply padding to the input for backpropagation
        x_padded = self.pad_input(x)
        
        batch_size, input_channels, input_length = x.shape
        delta_w = np.zeros_like(self.w, dtype=np.float64)
        delta_b = np.zeros_like(self.b, dtype=np.float64)
        
        # Initialize delta_x for the backpropagation of each sample in the batch
        delta_x = np.zeros_like(x_padded, dtype=np.float64)
        
        # Gradients for weights, biases, and input (accumulated over the batch)
        for i in range(self.output_channels):
            for b in range(batch_size):  # Process each sample in the batch
                for j in range(delta_a.shape[2]):
                    delta_b[i] += delta_a[b, i, j]
                    for c in range(input_channels):
                        delta_w[i, c] += delta_a[b, i, j] * x_padded[b, c, j:j + self.filter_size].sum()
                        delta_x[b, c, j:j + self.filter_size] += delta_a[b, i, j] * self.w[i, c]
        
        # Remove padding from the gradient w.r.t input (delta_x)
        delta_x = delta_x[:, :, self.padding:-self.padding] if self.padding > 0 else delta_x
        
        # Update weights and biases using gradient descent
        self.w -= learning_rate * delta_w
        self.b -= learning_rate * delta_b
        
        return delta_x


# Example usage with mini-batch:
x = np.array([[[1, 2, 3, 4], [2, 3, 4, 5]],  # Batch of 2 samples, 2 channels, 4 features
              [[3, 4, 5, 6], [4, 5, 6, 7]]])  # Batch of 2 samples, 2 channels, 4 features
w = np.ones((3, 2, 3))  # Filters: 3 output channels, 2 input channels, filter size 3
b = np.array([1, 2, 3])  # Biases: 3 output channels

# Create Conv1d layer with padding=1 (zero padding)
conv_layer = Conv1d(input_channels=2, output_channels=3, filter_size=3, padding=1)

# Forward pass
print("Forward pass output:")
print(output)

# Backpropagation (dummy delta_a for demonstration)
delta_a = np.array([[[10, 20], [15, 25], [30, 35]],  # Gradient w.r.t output for each sample
                    [[10, 20], [15, 25], [30, 35]]])  # Gradient w.r.t output for each sample
delta_x = conv_layer.backward(x, delta_a, learning_rate=0.01)

print("Backward pass:")
print("Gradient w.r.t input:")
print(delta_x)


Forward pass output:
[[ 9. 16. 22. 17.]
 [10. 17. 23. 18.]
 [11. 18. 24. 19.]]
Backward pass:
Gradient w.r.t input:
[[[135. 135.]
  [135. 135.]
  [  0.   0.]
  [  0.   0.]]

 [[135. 135.]
  [135. 135.]
  [  0.   0.]
  [  0.   0.]]]
